In [3]:
import pandas as pd
import pymysql

df = pd.read_csv("venta.csv")

df = df.rename(columns={
    "precio": "precio",
    "titulo": "titulo",
    "ubicacion": "ubicacion",
    "habitacionnum": "habitaciones",
    "baños": "banos",
    "superficie_construida": "superficie_construida",
    "superficie_util": "superficie_util",
    "descripcion_ampliada": "descripcion_ampliada",
    "tipo_vivienda": "tipo_vivienda",
    "link": "link"
})

# Crear tipo_vivienda a partir de las columnas booleanas de Pablo
lista_tipos_vivienda = ["piso", "atico", "estudio", "apartamento", "duplex", "chalet", "finca", "loft"]
df["tipo_vivienda"] = df[lista_tipos_vivienda].to_numpy().argmax(axis=1)
df["tipo_vivienda"] = df["tipo_vivienda"].apply(lambda val: lista_tipos_vivienda[val])

df["metros"] = df["superficie_construida"]

conexion = pymysql.connect(
    host="localhost",
    user="root",
    password="Destroyerl0",
    database="propiedades_de_ventas",
    charset="utf8mb4"
)

cursor = conexion.cursor()

for _, fila in df.iterrows():
    sql = """
    INSERT INTO anuncios (titulo, precio, ubicacion, habitaciones, banos, metros, tipo_vivienda, superficie_construida, superficie_util, descripcion_ampliada, link) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    datos = (
        fila["titulo"],
        float(fila["precio"]) if not pd.isna(fila["precio"]) else None,
        fila["ubicacion"],
        int(fila["habitaciones"]) if not pd.isna(fila["habitaciones"]) else None,
        int(fila["banos"]) if not pd.isna(fila["banos"]) else None,
        int(fila["metros"]) if not pd.isna(fila["metros"]) else None,
        fila["tipo_vivienda"],
        int(fila["superficie_construida"]) if not pd.isna(fila["superficie_construida"]) else None,
        int(fila["superficie_util"]) if not pd.isna(fila["superficie_util"]) else None,
        fila["descripcion_ampliada"],
        fila["link"]
    )
    cursor.execute(sql, datos)

conexion.commit()
cursor.close()
conexion.close()